In [1]:
from scipy.special import binom

In [2]:
def log2_int(n):
    p = 0
    i = 0
    while n > 0:
        if (n & 1<<i) != 0:
            n -= 1<<i
            p = i
        i += 1
    return p

def get_k_set_bit(n, k):
    i = 0
    while n > 0 and k > 0:
        if (n & 1<<i) != 0:
            n -= 1<<i
            k -= 1
        i += 1
    return i-1

In [3]:
def bin_enum(qss, sid, sq):
    s = 0
    t = 0
    while sq > 0 and qss > 0:
        o = binom(sq-1, qss-1)
        print(f"sid={sid}, sq={sq}, qss={qss}, o={o}, t={t}")
        if sid < o:
            s |= 1 << t
            qss -= 1
        else:
            sid = sid - o
        t += 1
        sq -= 1
    return s

def pos_enum(s, cid, qss):
    l = 0
    cid += 1
    while cid > 0:
        mtid = log2_int(cid)
        # print(f"mtid={mtid}, cid={cid}",)
        ntid = get_k_set_bit(s, qss-mtid)
        # print(f"ntid={ntid}",)
        l |= 1 << ntid
        # print(f"l=0b{l:b}")
        cid = cid - 2**mtid
    r = s - l 
    assert (l&s) == l
    return (l, r)

def pos_enum_pdep(s, cid, qss):
    _s = s
    l = 0
    cid += 1
    bb = 1
    while _s:
        if cid & bb:
            l |= _s & -_s
        _s &= _s - 1
        bb += bb
    return l, s-l

def dpsub_enum(s, cid, qss):
    l = s & (-s)
    while cid > 0:
        l = s & (l-s)
        cid -= 1
    return l, s-l


In [4]:
qss = 4
sq = 5
n = int(binom(sq, qss))
print(n, '\n')
for i in range(n):
    query = bin_enum(qss, i, sq)
    print(f"0b{query:0{sq}b}")
    for j in range(2**qss-2):
        l, r = pos_enum(query, j, qss)
        print(f"    0b{l:0{sq}b} 0b{r:0{sq}b}")
        assert (l|r) == query
        l_pdep, r_pdep = pos_enum_pdep(query, j, qss)
        print(f"        0b{l_pdep:0{sq}b} 0b{r_pdep:0{sq}b}")
        assert (l_pdep|r_pdep) == query
        l_dpsub, r_dpsub = dpsub_enum(query, j, qss)
        print(f"        0b{l_dpsub:0{sq}b} 0b{r_dpsub:0{sq}b}")
        assert (l_dpsub|r_dpsub) == query
        

5 

sid=0, sq=5, qss=4, o=4.0, t=0
sid=0, sq=4, qss=3, o=3.0, t=1
sid=0, sq=3, qss=2, o=2.0, t=2
sid=0, sq=2, qss=1, o=1.0, t=3
0b01111
    0b01000 0b00111
        0b00001 0b01110
        0b00001 0b01110
    0b00100 0b01011
        0b00010 0b01101
        0b00010 0b01101
    0b01100 0b00011
        0b00011 0b01100
        0b00011 0b01100
    0b00010 0b01101
        0b00100 0b01011
        0b00100 0b01011
    0b01010 0b00101
        0b00101 0b01010
        0b00101 0b01010
    0b00110 0b01001
        0b00110 0b01001
        0b00110 0b01001
    0b01110 0b00001
        0b00111 0b01000
        0b00111 0b01000
    0b00001 0b01110
        0b01000 0b00111
        0b01000 0b00111
    0b01001 0b00110
        0b01001 0b00110
        0b01001 0b00110
    0b00101 0b01010
        0b01010 0b00101
        0b01010 0b00101
    0b01101 0b00010
        0b01011 0b00100
        0b01011 0b00100
    0b00011 0b01100
        0b01100 0b00011
        0b01100 0b00011
    0b01011 0b00100
        0b01101 0b00010
    

In [5]:
log2_int(2)

1

In [5]:
for sq in range(2,11):
    for qss in range(2, sq+1):
        n_sets= int(binom(sq, qss))
        n_joins_per_set = 2**qss-2
        tot = n_sets * n_joins_per_set
        print(f"sq={sq:2d}, qss={qss:2d}: n_sets={n_sets:4d}, n_joins_per_set={n_joins_per_set:4d}, tot={tot}")
        

sq= 2, qss= 2: n_sets=   1, n_joins_per_set=   2, tot=2
sq= 3, qss= 2: n_sets=   3, n_joins_per_set=   2, tot=6
sq= 3, qss= 3: n_sets=   1, n_joins_per_set=   6, tot=6
sq= 4, qss= 2: n_sets=   6, n_joins_per_set=   2, tot=12
sq= 4, qss= 3: n_sets=   4, n_joins_per_set=   6, tot=24
sq= 4, qss= 4: n_sets=   1, n_joins_per_set=  14, tot=14
sq= 5, qss= 2: n_sets=  10, n_joins_per_set=   2, tot=20
sq= 5, qss= 3: n_sets=  10, n_joins_per_set=   6, tot=60
sq= 5, qss= 4: n_sets=   5, n_joins_per_set=  14, tot=70
sq= 5, qss= 5: n_sets=   1, n_joins_per_set=  30, tot=30
sq= 6, qss= 2: n_sets=  15, n_joins_per_set=   2, tot=30
sq= 6, qss= 3: n_sets=  20, n_joins_per_set=   6, tot=120
sq= 6, qss= 4: n_sets=  15, n_joins_per_set=  14, tot=210
sq= 6, qss= 5: n_sets=   6, n_joins_per_set=  30, tot=180
sq= 6, qss= 6: n_sets=   1, n_joins_per_set=  62, tot=62
sq= 7, qss= 2: n_sets=  21, n_joins_per_set=   2, tot=42
sq= 7, qss= 3: n_sets=  35, n_joins_per_set=   6, tot=210
sq= 7, qss= 4: n_sets=  35, n_

In [7]:
bin_enum(3, 5, 5)

sid=5, sq=5, qss=3, o=6.0, t=0
sid=5, sq=4, qss=2, o=3.0, t=1
sid=2.0, sq=3, qss=2, o=2.0, t=2
sid=0.0, sq=2, qss=2, o=1.0, t=3
sid=0.0, sq=1, qss=1, o=1.0, t=4


25

In [8]:
binom(-1,-1)

nan

In [56]:
def ceil_div(a,b):
    return (a+b-1)//b

In [55]:
n_parallel = 2
n_sets = 10
n_joins_per_set = 2**4 - 2
tot = n_sets * n_joins_per_set

print(n_parallel, n_sets, n_joins_per_set, tot)

if n_sets >= gpuqo_dpsub_n_parallel:
    n_joins_per_thread = n_joins_per_set
    n_sets_per_iteration = gpuqo_dpsub_n_parallel
else:
    n_joins_per_thread = ceil_div(tot, gpuqo_dpsub_n_parallel)
    n_sets_per_iteration = tot/n_joins_per_thread

print(n_sets_per_iteration, n_joins_per_thread)

offset = 0
id_offset = 0
n_iters = 0
while offset < tot:
    if (n_sets-offset/n_joins_per_set) >= n_sets_per_iteration:
        n_threads = n_sets_per_iteration*ceil_div(n_joins_per_set, n_joins_per_thread)
    else:
        n_threads = (n_sets-offset//n_joins_per_set)*(ceil_div(n_joins_per_set,n_joins_per_thread)

    splits_per_qs = ceil_div(n_joins_per_set,n_joins_per_thread)
    print(n_threads, splits_per_qs)
    for tid in range(n_threads):
        sid = (tid+id_offset) // splits_per_qs
        cid = ((tid+id_offset) % splits_per_qs)*n_joins_per_thread+1
        print("   ", sid, cid, cid + n_joins_per_thread)
    
    n_iters += 1
    offset += n_sets_per_iteration*n_joins_per_set
    id_offset += n_threads

2 10 14 140
1 7
2 2
    0 1 8
    0 8 15
2 2
    1 1 8
    1 8 15
2 2
    2 1 8
    2 8 15
2 2
    3 1 8
    3 8 15
2 2
    4 1 8
    4 8 15
2 2
    5 1 8
    5 8 15
2 2
    6 1 8
    6 8 15
2 2
    7 1 8
    7 8 15
2 2
    8 1 8
    8 8 15
2 2
    9 1 8
    9 8 15
